In [1]:
using gmsh
using GR 
using LinearAlgebra
using SparseArrays 
using Plots
using LaTeXStrings

using LinearAlgebra
using SparseArrays 
using StructArrays
using StaticArrays
using StaticRanges
using FastGaussQuadrature

using IterativeSolvers
using Preconditioners

using BenchmarkTools
using Profile
using ProfileView

using Plots 

In [2]:
#..1/4: initialize gmsh 
gmsh.initialize()

#..2/4: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc = .02
#..define four points via (x,y,z) coordinates 
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(1., 0,  0, lc, 2)
gmsh.model.geo.addPoint(1., 1., 0, lc, 3)
gmsh.model.geo.addPoint(0, 1., 0, lc, 4)
#..define four edges by connecting point labels pairwise  
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
#..define curved loop by connecting four edge labels  
gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
#..define surface by curved loop 
gmsh.model.geo.addPlaneSurface([1], 1)
#..give label to the surface 
#..syntax of the function being used is gmsh.model.setPhysicalName(dim, tag, name) 
gmsh.model.setPhysicalName(2, 1, "My surface")
#..synchronize model 
gmsh.model.geo.synchronize()

#..3/4: generate two-dimensional mesh 
gmsh.model.mesh.generate(2)
#..if true, write mesh to file for further processing 
if (true) gmsh.write("square.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..4/4: finalize gmsh 
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.191295s, CPU 0.171875s)
Info    : 3013 nodes 6028 elements
Info    : Writing 'square.msh'...
Info    : Done writing 'square.msh'


In [3]:
function area_triangle(x1,x2,x3,y1,y2,y3)
    x12 = x2 - x1; x13 = x3-x1;
    y12 = y2 - y1; y13 = y3-y1;
    area_id = x12*y13 - x13*y12; 
    area_id = abs(area_id)/2
    return area_id 
end

area_triangle (generic function with 1 method)

In [4]:
#..1/12: Finalize gmsh
gmsh.initialize()

#..2/12: Generate the mesh
gmsh.open("square.msh")

#..3/12 Get and sort the mesh nodes
#..Observe that although the mesh is two-dimensional,
#..the z-coordinate that is equal to zero is stored as well.
#..Observe that the coordinates are stored contiguously for computational
#..efficiency
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
nnodes = length(node_ids)
#..sort the node coordinates by ID, such that Node one sits at row 1
tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
sorted = sortslices(tosort , dims = 1);
node_ids = sorted[:,1]
xnode = sorted[:,2]
ynode = sorted[:,3]

#..4/12 Get the mesh elements
#..observe that we get all the two-dimensional triangular elements from the mesh
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
nelements = length(element_ids[1])
nelements


Info    : Reading 'square.msh'...
Info    : 9 entities
Info    : 3013 nodes
Info    : 6028 elements
Info    : Done reading 'square.msh'


5824

In [5]:
#..5/12 Define the source function and apply the source function to the local coordinates
sourcefunction(x,y) = 4*(x-x^2)+4*(y-y^2)
sourcefunction(x,y) = x+y
fvalues = map(sourcefunction, xnode, ynode)
display(xnode)
display(fvalues)

3013-element Vector{Float64}:
 0.0
 1.0
 1.0
 0.0
 0.01999999999996237
 0.03999999999992186
 0.05999999999987837
 0.07999999999984518
 0.09999999999981463
 0.1199999999997832
 ⋮
 0.9853589838485546
 0.01464101615143969
 0.06458576204092782
 0.04595514658377027
 0.04585515833654494
 0.9737471981092598
 0.9739466600872153
 0.06450687269860352
 0.9723948471707854

3013-element Vector{Float64}:
 0.0
 1.0
 2.0
 1.0
 0.01999999999996237
 0.03999999999992186
 0.05999999999987837
 0.07999999999984518
 0.09999999999981463
 0.1199999999997832
 ⋮
 1.9707179676971174
 1.000000000000001
 1.0267602687867703
 0.11567994927605127
 0.9763548577602976
 1.3132249019442157
 1.833045693992656
 0.10005837890977722
 1.6312067386716391

In [6]:
function genLocStiffMat(node_vec)
    #....retrieve the x and y coordinates of the local nodes of the current element
    xnode1 = xnode[node_vec[1]]; xnode2 = xnode[node_vec[2]]; xnode3 = xnode[node_vec[3]];
    ynode1 = ynode[node_vec[1]]; ynode2 = ynode[node_vec[2]]; ynode3 = ynode[node_vec[3]];
    area_id = area_triangle(xnode1,xnode2,xnode3,ynode1,ynode2,ynode3)
    
    Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]]\UniformScaling(1.);
    Emat[3,:] .= 0;
    Aloc = vec(area_id*(transpose(Emat)*Emat))
    iter = [node_vec,node_vec,node_vec]
    Iloc = vcat(iter...)
    Jloc = vec(vcat(transpose(iter)...))
    return Iloc, Jloc, Aloc
end

function genStiffMat()

    dofperelem = 9
    #..preallocate the memory for local matrix contributions 
    Avalues = zeros(Float64,dofperelem*nelements)
    I = zeros(Int64,length(Avalues))
    J = zeros(Int64,length(Avalues))
    display(length(I))


    for element_id in 1:nelements
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        nodes = Vector{Int64}([node1_id; node2_id; node3_id])

        
        Iloc, Jloc, Aloc = genLocStiffMat(nodes) 
        irng = mrange(dofperelem*element_id-dofperelem + 1,dofperelem*element_id) 

        I[irng] .= Iloc 
        J[irng] .= Jloc 
        Avalues[irng] .= Aloc  
    end

    A = sparse(I,J,Avalues)
    return A;
    
end
A = genStiffMat()

52416

3013×3013 SparseMatrixCSC{Float64, Int64} with 20685 stored entries:
⡻⣮⣹⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀⣖⣃⣔⡶⠄⠢⢬⣓⢀⠀⢀⠀⢀⠟⣿
⣷⣾⣿⢟⡀⠀⠀⢀⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⡀⢀⠀⠀⠀⣾⡲⠯⣧⣿⣤⡇⡺⣺⣯⢠⡀⢠⡥⣽⣷⣿
⠀⠀⠀⠈⠿⣧⡀⣸⠆⠀⣼⠇⡏⠀⣇⢿⢀⠀⠾⡦⠂⢀⠁⢀⡉⢁⡀⡒⣤⢦⢿⣿⣮⣸⢢⡴⢄⠂⠀⠀
⠀⠀⠀⢀⣀⣨⡿⣯⣀⠀⢸⠂⠺⡆⠆⣀⠀⡀⠀⠀⠀⠀⠄⢠⠐⣝⢠⣐⣰⣹⢞⣴⣽⢮⣭⣤⣑⠇⠀⠀
⠀⠀⠀⠀⠈⠁⠀⠘⠻⣦⣷⠂⣎⠗⡓⢻⠘⠁⢑⠐⠃⠀⠀⢸⣯⠶⡙⠃⣻⣚⣯⣳⡹⡻⢛⠯⢧⠠⠀⠀
⠀⠀⠀⠀⠶⠟⠲⠒⠹⠛⠿⣧⡞⢿⡟⡾⠈⠂⠒⠅⠀⠰⠆⠀⠚⣋⠃⢷⠾⣿⣟⣿⡿⢿⣖⣈⢞⠂⠀⠀
⠀⠀⠀⠠⠋⠉⠺⠦⢮⠝⣾⣍⣿⣿⣿⣿⣼⡆⢡⣢⠂⠀⡯⠠⠈⣼⢤⠼⣯⣿⣿⣯⡿⣿⡿⡯⠭⠅⠀⠀
⠀⠀⠀⠀⣭⣝⠈⢡⣽⣈⣻⡭⣿⣿⣿⣿⣿⣷⣿⣿⣂⡠⣯⠘⡍⡞⢳⣿⣾⣿⡷⣿⣿⣿⢽⣳⣟⡃⡖⠀
⠀⠀⠀⠀⠀⠐⠀⠠⠖⠀⠢⠀⠲⠿⢿⣿⣿⣿⣟⡿⠧⣻⢿⣡⢌⣙⣍⣯⣇⢾⣿⢿⣿⣿⢷⣿⡵⣮⣩⣅
⠀⠀⠀⢈⠺⡧⠀⠀⢑⠐⠜⠄⠡⣲⣿⣿⣿⡽⡿⣯⡏⢿⣿⣾⡟⡽⣋⡻⠟⣿⣷⣷⢮⣮⣽⣿⣽⣇⣒⡀
⠀⠀⠀⠀⠈⢀⠀⠀⠉⠀⢀⡀⠈⠀⠈⡸⣭⣣⣯⣍⢻⣶⡿⣿⣷⣵⡛⣿⣭⣋⣩⣹⣑⣿⣽⣭⡽⡿⣦⡔
⠀⡀⣠⣤⠁⢀⠀⣁⣀⣀⠈⠁⠋⡋⣋⠛⠟⣳⣻⣿⣿⣯⡿⣯⣟⣮⣋⣿⣻⣼⠟⢻⣝⣯⡻⢿⡟⢟⣴⣶
⢠⢬⡼⡎⠇⢈⣔⢤⢫⡟⡾⢠⣂⣤⣣⠭⣆⢱⣟⡭⢝⣿⡻⣽⣿⣿⢅⣛⣽⣿⡿⣝⣿⣪⣽⣻⡏⣿⣤⢧
⢉⢼⣭⣿⢠⠨⢀⢲⠷⠈⢭⣄⣀⡗⣽⣶⡧⣽⣯⡸⣿⣬⣯⣼⣥⢱⡵⣯⡿⡼⣏⡞⡲⣷⣼⣿⣮⣬⣿⣿
⠘⠏⠤⠿⠠⣟⣔⣺⣻⢺⣾⣧⣯⣿⣾⣿⣩⣝⣿⣥⡧⢻⣛⣾⣷⣿⣛⡯⣿⢟⣷⣾⣟⡭⣿⡩⣿⣏⠿⢿
⡈⣆⣺⣪⣿⣷⢚⣵⢯⣻⣿⣽⡿⣿⣽⣯⣿⣟⢽⣿⣇⣺⣿⣁⣟⢯⣫⠽⣹⣿⣿⣿⣷⡿⣿⣿⣷⣓⢸⣒
⠙⢘⠋⣛⣊⣻⡳⣟⣷⡪⣿⣏⣿⣯⣿⣿⣿⣿⡪⣷⣵⣼⡷⣽⡻⣻⢼⣮⡟⡽⣽⡿⣿⣿⣿⣿⣿⣽⢸⠧
⠀⢀⠀⣈⢈⡶⠃⣿⡿⡔⡘⢹⡿⡯⢷⣳⣽⣷⣷⣿⡗⣿⣿⣎⣷⣻⣶⣿⡟⡻⣿⣿⣿⣿⣿⣿⣧⣉⢃⡄
⠀⢀⣅⣯⠠⠑⠵⠜⠉⡓⠺⠑⠇⠇⠿⠹⡱⣯⠷⢿⣷⡯⣿⢍⣯⣭⡊⣿⡿⢿⢽⢻⣟⣿⡍⢻⣻⣾⡤⣁
⣿⣥⣽⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠉⠇⢾⠘⠸⢈⠿⢰⣿⠤⣟⣿⣿⣿⣇⢲⢲⠶⡖⠉⠴⠄⢫⣿⣿

In [7]:
function genLocVec(node_vec, fvalues)
    xnode1 = xnode[node_vec[1]]; xnode2 = xnode[node_vec[2]]; xnode3 = xnode[node_vec[3]];
    ynode1 = ynode[node_vec[1]]; ynode2 = ynode[node_vec[2]]; ynode3 = ynode[node_vec[3]];
    area_id = area_triangle(xnode1,xnode2,xnode3,ynode1,ynode2,ynode3)
    Iloc = node_vec
    floc = area_id/3*fvalues[node_vec]
    return Iloc, floc
end

function genVector(fvalues)
    
    
    
    #..initialize global vector  
    f = zeros(Float64,nnodes)

    for element_id = 1:nelements #..loop over number of elements..
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        nodes = Vector{Int64}([node1_id; node2_id; node3_id])
        Iloc, floc = genLocVec(nodes, fvalues)
        f[Iloc] .+= floc 
    end
   
    return f; 
end

#f = genVector(mesh,fsource)
#println(f)
f = genVector(fvalues)

3013-element Vector{Float64}:
 0.0
 9.760677434282912e-5
 0.00019521354868654698
 9.760677434333387e-5
 3.1430448234920964e-6
 6.101892274413435e-6
 8.99964184233497e-6
 1.3979693507906273e-5
 1.61364333063929e-5
 1.94331769839081e-5
 ⋮
 0.0004587275500742443
 0.0002285412438103891
 0.00023711702266664798
 2.529902488875172e-5
 0.0002157424895737907
 0.0002563335736119266
 0.0003663873345099726
 2.185161384492055e-5
 0.00031668149404124506

In [8]:
#..8/12 Handle the boundary conditions
#..retrieve boundary nodes by loop over corner point and boundary edges
node_ids1=[]; node_ids2=[]; node_ids3=[]; node_ids4=[]; 
node_ids5=[]; node_ids6=[]; node_ids7=[]; node_ids8=[]; 
node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,1)
node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,2)
node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
node_ids4, node_coord, _ = gmsh.model.mesh.getNodes(0,4)
node_ids5, node_coord, _ = gmsh.model.mesh.getNodes(1,1)
node_ids6, node_coord, _ = gmsh.model.mesh.getNodes(1,2)
node_ids7, node_coord, _ = gmsh.model.mesh.getNodes(1,3)
node_ids8, node_coord, _ = gmsh.model.mesh.getNodes(1,4)
bnd_node_ids = union(node_ids1,node_ids2,node_ids3,node_ids4,node_ids5,node_ids6,node_ids7,node_ids8)
A[bnd_node_ids,:] .= 0;
A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
f[bnd_node_ids] .= 0;

#..9/12 Make A sparse and M diagonal 
#A = sparse(A)
#M = Diagonal(M)

#..10/12 Compute the numerical solution
u = A\f

#..10/12: Finalize gmsh
gmsh.finalize()

#..11/12 Plot the numerical solution
GR.trisurf(xnode,ynode,u)